In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)

plt.style.use("classic")
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]


if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

/Users/e1211913/miniconda3/envs/trading/lib/python3.12/site-packages/telegram/utils/request.py:49: UserWarning: python-telegram-bot is using upstream urllib3. This is allowed but not supported by python-telegram-bot maintainers.
  warnings.warn(


logging mode: dev


In [2]:
%%html
<style>
.dataframe {
    font-size: 9pt; /* Adjust font size as needed */
}
</style>

In [7]:
df = pth.load_instruments(base_path=base_data_path)
df.keys()

dict_keys(['6B', 'MES'])

In [17]:
df["6B"].close

timestamp
2024-5-19  23:00:00.000000    1.2706
2024-5-19  23:01:00.000000    1.2707
2024-5-19  23:02:00.000000    1.2707
2024-5-19  23:03:00.000000    1.2708
2024-5-19  23:04:00.000000    1.2708
                               ...  
2025-3-14  18:07:00.000000    1.2928
2025-3-14  18:08:00.000000    1.2928
2025-3-14  18:09:00.000000    1.2927
2025-3-14  18:11:00.000000    1.2929
2025-3-14  18:12:00.000000    1.2929
Name: close, Length: 268474, dtype: float64

In [20]:
# TODO: continue here when you figure out how to use data
# df.vbt.plot()

In [8]:
df.data['ADAUSDT'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2773 entries, 2017-08-17 00:00:00+00:00 to 2025-03-20 00:00:00+00:00
Freq: D
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open                2530 non-null   float64
 1   High                2530 non-null   float64
 2   Low                 2530 non-null   float64
 3   Close               2530 non-null   float64
 4   Volume              2530 non-null   float64
 5   Quote volume        2530 non-null   float64
 6   Trade count         2530 non-null   float64
 7   Taker base volume   2530 non-null   float64
 8   Taker quote volume  2530 non-null   float64
dtypes: float64(9)
memory usage: 216.6 KB


In [9]:
vbt.indicator("talib:RSI") # fast indicator, but no plotting

vectorbtpro.indicators.factory.talib.RSI

In [10]:
# tiny bit slower + plotting
vbt.RSI

vectorbtpro.indicators.custom.rsi.RSI

In [11]:
rsi = vbt.RSI.run(df.get('Close', 'ADAUSDT'))
rsi.rsi

Open time
2017-08-17 00:00:00+00:00          NaN
2017-08-18 00:00:00+00:00          NaN
2017-08-19 00:00:00+00:00          NaN
2017-08-20 00:00:00+00:00          NaN
2017-08-21 00:00:00+00:00          NaN
                               ...    
2025-03-16 00:00:00+00:00    45.195270
2025-03-17 00:00:00+00:00    46.154043
2025-03-18 00:00:00+00:00    45.088826
2025-03-19 00:00:00+00:00    48.484367
2025-03-20 00:00:00+00:00    47.121329
Freq: D, Name: Close, Length: 2773, dtype: float64

In [12]:
entries, exits = rsi.rsi_crossed_below(30), rsi.rsi_crossed_above(70)
entries.value_counts(), exits.value_counts()

(Close
 False    2735
 True       38
 Name: count, dtype: int64,
 Close
 False    2724
 True       49
 Name: count, dtype: int64)

In [13]:
def plot_rsi(rsi, entries, exits):
    fig = rsi.plot()
    entries.vbt.signals.plot_as_entries(rsi.rsi, fig=fig)
    exits.vbt.signals.plot_as_exits(rsi.rsi, fig=fig)
    fig.show()

plot_rsi(rsi, entries=entries, exits=exits)

In [14]:
entries, exits = entries.vbt.signals.clean(exits)
plot_rsi(rsi, entries=entries, exits=exits)

In [15]:
entries.vbt.signals.total(), exits.vbt.signals.total()

(np.int64(12), np.int64(12))

In [16]:
ranges = entries.vbt.signals.between_ranges(target=exits)
ranges.duration.values

array([273, 188,  45,  61,  18, 117, 254,  36,  34,  67, 209,  26])

In [17]:
ranges.duration.mean(), ranges.duration.mean(wrap_kwargs=dict(to_timedelta=True))

(np.float64(110.66666666666667), Timedelta('110 days 16:00:00'))

In [18]:
pf = vbt.Portfolio.from_signals(df.get('Close', 'ADAUSDT'), entries, exits, size=100, size_type="value", init_cash="auto")
pf.stats()

Start Index                   2017-08-17 00:00:00+00:00
End Index                     2025-03-20 00:00:00+00:00
Total Duration                       2773 days 00:00:00
Start Value                                  192.635464
Min Value                                     83.008442
Max Value                                    342.810368
End Value                                    342.810368
Total Return [%]                              77.958077
Benchmark Return [%]                         199.010717
Position Coverage [%]                         47.890371
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              62.950076
Max Drawdown Duration                1186 days 00:00:00
Total Orders                                         24
Total Fees Paid                                     0.0
Total Trades                                         12
Win Rate [%]                                  58.333333
Best Trade [%]                                97

In [19]:
pf.plot(settings=dict(bm_returns=False))

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e61109fd-5df8-47cf-bd53-bbf67d9cd8ac',
              'x': array([datetime.datetime(2017, 8, 17, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 18, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 19, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2025, 3, 18, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2025, 3, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2025, 3, 20, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([   nan,    nan,    nan, ..., 0.7017, 0.7436, 0.7254])

### Multiple Params Backtest

In [20]:
windows = list(range(8, 21))
window_types = ['simple', 'exp', 'wilder']
lower_ths = list(range(20, 31))
upper_ths = list(range(70, 81))

In [21]:
rsi = vbt.RSI.run(
    df.get('Open', 'ADAUSDT'),
    window=windows,
    wtype=window_types,
    param_product=True,
)
rsi.rsi

rsi_window                        8                                9   \
rsi_wtype                     simple        exp     wilder     simple   
Open time                                                               
2017-08-17 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-18 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-19 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-20 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-21 00:00:00+00:00        NaN        NaN        NaN        NaN   
...                              ...        ...        ...        ...   
2025-03-16 00:00:00+00:00  37.664588  49.698413  47.701166  28.825245   
2025-03-17 00:00:00+00:00  34.214668  38.141852  42.826189  32.889561   
2025-03-18 00:00:00+00:00  49.169031  43.244494  44.766169  36.724190   
2025-03-19 00:00:00+00:00  57.216981  38.168364  42.690265  46.209524   
2025-03-20 00:00:00+00:00  54.875622  55.841616  49.752195  64.277275   

rsi_window                                              10             \
rsi_wtype                        exp     wilder     simple        exp   
Open time                                                               
2017-08-17 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-18 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-19 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-20 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-21 00:00:00+00:00        NaN        NaN        NaN        NaN   
...                              ...        ...        ...        ...   
2025-03-16 00:00:00+00:00  48.803054  47.729026  24.443445  48.280402   
2025-03-17 00:00:00+00:00  39.131522  43.512250  25.942721  39.985294   
2025-03-18 00:00:00+00:00  43.351437  45.151597  35.354124  43.549985   
2025-03-19 00:00:00+00:00  39.102218  43.375761  35.047674  39.916270   
2025-03-20 00:00:00+00:00  54.036536  49.322200  53.613666  52.706340   

rsi_window                                   11  ...         17         18  \
rsi_wtype                     wilder     simple  ...     wilder     simple   
Open time                                        ...                         
2017-08-17 00:00:00+00:00        NaN        NaN  ...        NaN        NaN   
2017-08-18 00:00:00+00:00        NaN        NaN  ...        NaN        NaN   
2017-08-19 00:00:00+00:00        NaN        NaN  ...        NaN        NaN   
2017-08-20 00:00:00+00:00        NaN        NaN  ...        NaN        NaN   
2017-08-21 00:00:00+00:00        NaN        NaN  ...        NaN        NaN   
...                              ...        ...  ...        ...        ...   
2025-03-16 00:00:00+00:00  47.769085  29.692404  ...  47.779802  52.286454   
2025-03-17 00:00:00+00:00  44.036824  22.338677  ...  45.587364  52.054402   
2025-03-18 00:00:00+00:00  45.460625  28.104727  ...  46.373589  52.572484   
2025-03-19 00:00:00+00:00  43.904308  33.797715  ...  45.497694  52.442050   
2025-03-20 00:00:00+00:00  49.044380  42.061856  ...  48.251692  52.955613   

rsi_window                                              19             \
rsi_wtype                        exp     wilder     simple        exp   
Open time                                                               
2017-08-17 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-18 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-19 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-20 00:00:00+00:00        NaN        NaN        NaN        NaN   
2017-08-21 00:00:00+00:00        NaN        NaN        NaN        NaN   
...                              ...        ...        ...        ...   
2025-03-16 00:00:00+00:00  47.749218  47.759980  52.341281  47.769085   
2025-03-17 00:00:00+00:00  43.791849  45.677488  50.757311  44.036824   
2025-03-18 00:00:00+00:00  45.315091  46.421348  52.482974

In [22]:
from itertools import product

lower_ths_prod, upper_ths_prod = zip(*product(lower_ths, upper_ths))
len(lower_ths_prod), len(upper_ths_prod)

(121, 121)

In [28]:
lower_ths_index = vbt.Param(lower_ths_prod, name='lower_th')
upper_ths_index = vbt.Param(upper_ths_prod, name='upper_ths')

entries = rsi.rsi_crossed_below(lower_ths_index)
exits = rsi.rsi_crossed_above(upper_ths_index)

entries.columns.shape, exits.columns.shape

((4719,), (4719,))

In [30]:
pf = vbt.Portfolio.from_signals(
    df.get('Close', 'ADAUSDT'),
    entries=entries,
    exits=exits,
    size=100,
    size_type="value",
    init_cash="auto",
)
pf

In [34]:
pf.stats(['total_return', 'total_trades', 'win_rate', 'expectancy'], agg_func=None)

Total Return [%]  Total Trades  \
lower_th upper_ths rsi_window rsi_wtype                                   
20       70        8          simple           -18.025853            47   
                              exp              -28.504551            40   
                              wilder            -7.181458            16   
                   9          simple            65.586850            44   
                              exp              -52.629968            35   
...                                                   ...           ...   
30       80        19         exp              -24.881817            11   
                              wilder           259.945626             3   
                   20         simple           115.132125            10   
                              exp               -1.963360            10   
                              wilder           687.125717             3   

                                         Win Rate [%]  Expectancy  
lower_th upper_ths rsi_window rsi_wtype                            
20       70        8          simple        54.347826   -0.855465  
                              exp           52.500000   -1.859529  
                              wilder        37.500000   -0.874305  
                   9          simple        65.909091    2.085653  
                              exp           51.428571   -3.858252  
...                                               ...         ...  
30       80        19         exp           50.000000   -3.855613  
                              wilder        33.333333  113.042681  
                   20         simple        44.444444   21.108935  
                              exp           55.555556    0.814338  
                              wilder        33.333333  298.810694  

[4719 rows x 4 columns]

In [59]:
pf.stats(metrics=['total_return'], split_columns=['lower_th', 'upper_th'], agg_func=None).groupby(
    ['lower_th', 'upper_ths']
).sum().vbt.heatmap()

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'type': 'heatmap',
              'uid': 'e995bf75-5ad6-412e-954f-7b8d8a248f59',
              'x': array(['Total Return [%]'], dtype=object),
              'y': [(20, 70), (20, 71), (20, 72), (20, 73), (20, 74), (20, 75),
                    (20, 76), (20, 77), (20, 78), (20, 79), (20, 80), (21, 70),
                    (21, 71), (21, 72), (21, 73), (21, 74), (21, 75), (21, 76),
                    (21, 77), (21, 78), (21, 79), (21, 80), (22, 70), (22, 71)

In [70]:
# single combination of params
column_comb = (22, 80, 20, 'simple')
pf.plot_value(column=column_comb)

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'f9241a26-6146-49aa-a31a-d7167fcd0642',
              'x': array([datetime.datetime(2017, 8, 17, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 18, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2017, 8, 19, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2025, 3, 18, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2025, 3, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2025, 3, 20, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([154.18604651, 154.18604651, 154.18604651, ..., 154.18604651,


In [80]:
pf[column_comb].stats()

Start Index                   2017-08-17 00:00:00+00:00
End Index                     2025-03-20 00:00:00+00:00
Total Duration                       2773 days 00:00:00
Start Value                                  154.186047
Min Value                                     57.547684
Max Value                                    690.389803
End Value                                    632.306989
Total Return [%]                             310.093522
Benchmark Return [%]                         199.010717
Position Coverage [%]                         43.635052
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              65.895881
Max Drawdown Duration                 691 days 00:00:00
Total Orders                                         15
Total Fees Paid                                     0.0
Total Trades                                          8
Win Rate [%]                                  57.142857
Best Trade [%]                               488

In [60]:
# TODO: continue here with more exmaples